In [21]:
%load_ext autoreload
%autoreload 2

import os
import zarr
import dask.array as da
import napari
import sys
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from ete3 import Tree

sys.path.append('../libraries')
from family_graph import build_Newick_tree, render_tree_view
from cells_database import Base, CellDB, TrackDB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.environ['NAPARI_PERFMON'] = '0'

In [3]:
labels_zarr_path = r'D:\kasia\tracking\E6_exp\code\tests\example_track_labels.zarr'
labels = da.from_zarr(labels_zarr_path)

In [4]:
# display the labels
viewer = napari.Viewer()
labels_layer = viewer.add_labels(labels,name='Labels')

In [5]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\cells_database_2tables.db'
engine = create_engine(f'sqlite:///{new_db_path}')

In [6]:
import pyqtgraph as pg

def build_lineage_widget(t_max):

    '''
    Builds pyqt widget to display lineage tree.
    '''
    
    global plot_widget

    plot_widget = pg.GraphicsLayoutWidget()
    plot_view = plot_widget.addPlot(title="Lineage tree", labels={"bottom": "Time"})
    plot_view.hideAxis("left")
    plot_view.setXRange(0, t_max)

    return plot_widget

In [7]:
# create a plot widget
t_max = viewer.dims.range[0][1]
plot_widget = build_lineage_widget(t_max)

# add lineage graph
viewer.window.add_dock_widget(plot_widget,area='bottom',name='family')

In [22]:
def update_lineage_display(event):

    global plot_widget # it may be possible to extract from napari, at the moment a global thing
    global viewer
    global time_line
    
    # clear the widget
    plot_view = plot_widget.getItem(0,0)
    plot_view.clear()

    # get fan active label
    active_label = int(viewer.layers['Labels'].selected_label)

    # check if the label is in the database
    with Session(engine) as session:
        
        query = session.query(TrackDB).filter(TrackDB.track_id == active_label).first()

    # actions based on finding the label in the database
    if query is not None:

        # get the root value
        root = query.root

        # update viewer status
        viewer.status = f'Family of track number {root}.'

        # buid the tree
        tree = build_Newick_tree(engine, root)

        # update the widget with the tree
        plot_view = render_tree_view(plot_view,tree,viewer)

        
    else:
        viewer.status = 'Error - no such label in the database.'
    

In [23]:
# connect lineage graph update
labels_layer.events.selected_label.connect(update_lineage_display)

<function __main__.update_lineage_display(event)>

### Test

In [11]:
my_root = 1

plot_view = plot_widget.getItem(0,0)
plot_view.clear()

# buid the tree
tree = build_Newick_tree(engine, my_root)

In [24]:
# update the widget with the tree
plot_view = render_tree_view(plot_view,tree,viewer)